In [1]:
import numpy as np
from scipy.spatial import Voronoi
from scipy.spatial import distance
from sympy import symbols, Eq, solve
import math
import itertools
from scipy.spatial import Delaunay

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
grid = np.array([
    [2, 0, 0, 0],
    [1, 1, 0, 0],
    [1, 0, 1, 0],
    [1, 0, 0, 1]
], dtype = float)

In [4]:
points_count = 8

coords4 = []
digits = range(- (points_count // 4), points_count - (points_count // 4))

for var in itertools.product(digits, repeat=4):
    coords4.append((grid.T).dot(var).tolist())
    
#coords4 # координаты центров

In [5]:
# строим диаграмму Вороного

vor4 = Voronoi(coords4)

In [ ]:
# вершины диаграммы Вороного

vor4.vertices

In [ ]:
# регионы

vor4.regions

In [8]:
# список конечных областей Вороного (-1 в координатах означает, что область бесконечна)

count = 0
vor4.regions_finite = []
for i in vor4.regions:
    if -1 not in i:
        vor4.regions_finite.append(i)
        count += 1
        
print(count)

1297


In [9]:
# максимальный размер региона (по числу вершин)

len_r_max = 24 # размер максимального региона
count_r = 0 # количество таких регионов

for i in range(len(vor4.regions)):
    len_r = len(vor4.regions[i])
    
    if len_r >= len_r_max:
        len_r_max = len_r
        count_r += 1 # считаем количество таких регионов
        
print('количество вершин =', len_r_max, 'количество =', count_r)

количество вершин = 24 количество = 1296


In [10]:
# находим суммарные расстояния от вершин многогранников до (0, 0, 0, 0) (sum_dist), затем находим минимальное 
# значение. Таким образом находим ближайший центральный многогранник

sum_dist_min = 1000

for i in range(1, len(vor4.regions)):
    len_r = len(vor4.regions[i])

    if -1 not in vor4.regions[i]: # регион должен быть конечным
        l = 0
        
        # для i региона перебираем все его вершины и для каждой ищем расстояние до [0.0, 0.0, 0.0, 0.0]
        for j in range(len_r):
            l += distance.euclidean(vor4.vertices[vor4.regions[i][j]], [0.0, 0.0, 0.0, 0.0])

        if l < sum_dist_min:
            sum_dist_min = l
            v_min = i

        
print ('суммарное расстояние =', sum_dist_min, 'индекс центрального региона =', v_min)

суммарное расстояние = 24.0 индекс центрального региона = 1320


In [11]:
# центральный регион

central = vor4.vertices[vor4.regions[v_min]]
central

array([[ 0.5, -0.5, -0.5, -0.5],
       [ 0.5, -0.5, -0.5,  0.5],
       [ 0.5, -0.5,  0.5,  0.5],
       [ 0. ,  0. ,  0. ,  1. ],
       [ 0.5, -0.5,  0.5, -0.5],
       [ 0. , -1. ,  0. ,  0. ],
       [-0.5, -0.5, -0.5,  0.5],
       [ 0. ,  0. ,  1. ,  0. ],
       [-0.5,  0.5,  0.5,  0.5],
       [-0.5,  0.5, -0.5,  0.5],
       [ 0. ,  0. , -1. ,  0. ],
       [-0.5, -0.5, -0.5, -0.5],
       [-0.5,  0.5, -0.5, -0.5],
       [-1. ,  0. ,  0. ,  0. ],
       [-0.5, -0.5,  0.5, -0.5],
       [-0.5,  0.5,  0.5, -0.5],
       [ 0. ,  0. ,  0. , -1. ],
       [-0.5, -0.5,  0.5,  0.5],
       [ 0.5,  0.5,  0.5,  0.5],
       [ 0.5,  0.5, -0.5,  0.5],
       [ 1. ,  0. ,  0. ,  0. ],
       [ 0.5,  0.5,  0.5, -0.5],
       [ 0.5,  0.5, -0.5, -0.5],
       [ 0. ,  1. ,  0. ,  0. ]])

In [12]:
# индексы вершин центрального региона

vor4.regions[v_min]

[3194,
 3196,
 3922,
 3932,
 5504,
 6827,
 6828,
 7422,
 7423,
 7437,
 7441,
 7720,
 7721,
 7722,
 7729,
 7730,
 7733,
 7894,
 8159,
 8164,
 8165,
 8166,
 8167,
 8174]

In [ ]:
# список многоугольников

vor4.ridge_vertices

In [13]:
edge_central = []
for i in range(len(vor4.ridge_vertices)):
    count = 0

    for j in range(len(vor4.ridge_vertices[i])):
        if vor4.ridge_vertices[i][j] in vor4.regions[v_min]:
            count += 1 

    if count == len(vor4.ridge_vertices[i]):
        edge_central.append(vor4.ridge_vertices[i])
        
# перевожу индексы вершин в координаты
edge_central_coords = []
for i in range(len(edge_central)):
    r = []
    for j in range(len(edge_central[i])):
        r.append(vor4.vertices[edge_central[i][j]])
    edge_central_coords.append(np.array(r))
    
edge_central_coords

[array([[-0.5, -0.5, -0.5, -0.5],
        [-0.5,  0.5, -0.5, -0.5],
        [-1. ,  0. ,  0. ,  0. ],
        [-0.5, -0.5,  0.5, -0.5],
        [-0.5,  0.5,  0.5, -0.5],
        [ 0. ,  0. ,  0. , -1. ]]),
 array([[-0.5, -0.5, -0.5,  0.5],
        [-0.5,  0.5, -0.5,  0.5],
        [ 0. ,  0. , -1. ,  0. ],
        [-0.5, -0.5, -0.5, -0.5],
        [-0.5,  0.5, -0.5, -0.5],
        [-1. ,  0. ,  0. ,  0. ]]),
 array([[ 0.5, -0.5, -0.5, -0.5],
        [ 0.5, -0.5,  0.5, -0.5],
        [ 0. , -1. ,  0. ,  0. ],
        [-0.5, -0.5, -0.5, -0.5],
        [-0.5, -0.5,  0.5, -0.5],
        [ 0. ,  0. ,  0. , -1. ]]),
 array([[ 0.5, -0.5, -0.5, -0.5],
        [ 0.5, -0.5, -0.5,  0.5],
        [ 0. , -1. ,  0. ,  0. ],
        [-0.5, -0.5, -0.5,  0.5],
        [ 0. ,  0. , -1. ,  0. ],
        [-0.5, -0.5, -0.5, -0.5]]),
 array([[ 0. ,  0. ,  0. ,  1. ],
        [-0.5, -0.5, -0.5,  0.5],
        [-0.5,  0.5,  0.5,  0.5],
        [-0.5,  0.5, -0.5,  0.5],
        [-1. ,  0. ,  0. ,  0. ],
      

In [14]:
# составляем список граней многогранника

list_faces = []

# берем каждый 3х мерный многогранник
for m in edge_central_coords:
    
    # составляем список ребер (расстояние между вершинами, соединенными ребрами = 1)
    list_edges = []
    for i in range(len(m)-1):
        for j in range(i+1, len(m)):
            k = []
            d = distance.euclidean(m[i], m[j])
            if d == 1:
                k.append(i)
                k.append(j)
                list_edges.append(k)
    # перебираем все комбинации ребер и ищем грани (если количество задействованных вершин в комбинации из 3х
    #  ребер = 3, то это грань)                       
    digits = list_edges
    list_faces_one = []  
    for var in itertools.combinations(digits, r=3):
        set_vet = set(var[0] + var[1] + var[2])
        set_vet_coord = []
        if len(set_vet) == 3:
            for i in range(len(set_vet)):
                set_vet_coord.append(m[list(set_vet)[i]])
            list_faces_one.append(set_vet_coord)
               
    list_faces.append(list_faces_one)
    
print('количество граней в одном зх мерном многограннике =', len(list_faces[1]))

количество граней в одном зх мерном многограннике = 8


In [15]:
# диаметр многогранника (ищем максимальное расстояние между вершинами)

max_len = 0

for i in range(len(central) - 1):
    for j in range(i+1, len(central)):
        len_v = distance.euclidean(central[i], central[j])
        if len_v > max_len:
            max_len = len_v
            u1 = i
            u2 = j
               
print('диаметр =', max_len, 'индексы вершин:', u1, ',',  u2) 
# u1, u2 - индексы вершин, расстояние между которыми максимально

диаметр = 2.0 индексы вершин: 0 , 8


In [16]:
# координаты проекции

# coord1, coord2, coord3 - координаты 3 точек, d - точка, проекцию которой надо найти

def coords_proj(coord1, coord2, coord3, d):
    
    M = []        
    M.append(coord1)
    
    l1 = []
    l2 = []
    
    for i in range(4):
        l1.append(coord2[i] - coord1[i])
        l2.append(coord3[i] - coord1[i])
        
    M.append(l1)
    M.append(l2)
    H1 = (np.array(M[1])).dot((np.array(M).T))
    K1 = np.array(l1).dot(d)
    H2 = (np.array(M[2])).dot((np.array(M).T))
    K2 = np.array(l2).dot(d)

    # Определение переменных
    x1, x2 = symbols('x1 x2')

    # Определение системы уравнений
    equations = [
        Eq(H1[0] + H1[1]*x1 + H1[2]*x2, K1),
        Eq(H2[0] + H2[1]*x1 + H2[2]*x2, K2)
    ]

    # Решение системы
    symbolic_solution = solve(equations, (x1, x2))

    # Вывод решения
    symbolic_solution
    
    coords = [1]
    coords.append(symbolic_solution[x1])
    coords.append(symbolic_solution[x2])
    
    
    return np.array(M).T.dot(coords)

In [17]:
# расстояние от точки до отрезка

import math

def distance_point_to_edge(p, a, b):
    
    # скалярное умножение 2х векторов
    def dot_product(v, w):
        sum_coord = 0
        for i in range(len(v)):
            sum_coord += v[i]*w[i]
        return sum_coord
    
    # разность 2х векторов
    def vector(v, w):
        list_dif = []
        for i in range(len(v)):
            k = w[i] - v[i]
            list_dif.append(k)
        return list_dif
    
    # длина вектора
    def magnitude(v):
        sum_sqr = 0
        for i in range(len(v)):
            sum_sqr += v[i]*v[i]
        return math.sqrt(sum_sqr)
    
    # умножение вектора на константу
    def scale(v, const):
        list_const = []
        for i in range(len(v)):
            list_const.append(v[i]*const)
        return list_const

    # сумма 2х векторов
    def add(v, w):
        sum_add = []
        for i in range(len(v)):
            sum_add.append(v[i]+w[i])
        return sum_add

    ab = vector(a, b)
    ap = vector(a, p)

    ab_magnitude = magnitude(ab)
    ap_dot_ab = dot_product(ap, ab)

    if ap_dot_ab <= 0:
        
        # Проекция точки на отрезок находится вне отрезка и ближе к точке a
        return magnitude(ap)
    
    if ap_dot_ab >= ab_magnitude*ab_magnitude:
        
        # Проекция точки на отрезок находится вне отрезка и ближе к точке b
        dist_b = 0
        for i in range(len(b)):
            dist_b += ((p[i] - b[i])**2)
        return math.sqrt(dist_b)

    # Проекция точки на отрезок находится внутри отрезка
    projection = add(a, scale(ab, ap_dot_ab/ab_magnitude**2))
    return magnitude(vector(p, projection))

In [18]:
# строим триангуляцию Делоне, используя ее будем определять находится ли проекция внутри многогранника

delaunay = Delaunay(central)

In [19]:
# 6
# решетка в обычном базисе

# ищет для подрешетки запрещенное расстояние
def ch_number_forb_segm(grid, sub_grid_0):
    
    # подрешетка в обычном базисе
    sub_grid = ((sub_grid_0).dot(grid))
    
    # подрешетка в  обычных координатах 
    points_count = 4
    coords4_1 = []
    
    digits = range(- (points_count // 4), points_count - (points_count // 4))
    
    for var in itertools.product(digits, repeat=4):
        coords4_1.append((sub_grid.T).dot(var).tolist())
    
    # ищем координаты, которые есть в решетке и подрешетке   
    common_coord = []
    for i in coords4:
        if i in coords4_1:
            common_coord.append(i)
    
    # расчитывам расстояние от [0.0, 0.0, 0.0, 0.0] до остальных точек и ищем минимальное расстояние
    min_dist = float('inf')
    for i in range(len(common_coord)):
        dist_centr = distance.euclidean(common_coord[i], [0.0, 0.0, 0.0, 0.0])
        
        if common_coord[i] != [0, 0, 0, 0] and dist_centr < min_dist:
            
            min_dist = dist_centr 
            min_dist_i = i #расстояние от [0.0, 0.0, 0.0, 0.0] до ближайшего другого центра
    
    # находим точку s (середину между центрами многограников)
    s = np.array(common_coord[min_dist_i], float) * 0.5

    min_dist_s = float('inf') #минимальное расстояние до точки s 
     
    # перебирем все грани
    for i in range(len(list_faces)):

        for j in range(len(list_faces[i])):

            # находим координаты проекции на грань и расстояние от точки до проекции
            f_coords = coords_proj(list_faces[i][j][0], list_faces[i][j][1], list_faces[i][j][2], s)
            f = distance.euclidean(s, np.array(f_coords, dtype = 'float'))

            # если расстояние меньше, то проверяем находится ли точка проекции в триангуляции
            # если да, то приравниваем минимальное расстояние к расстоянию между точкой и ее проекцией
            if f < min_dist_s:            
                simplex = delaunay.find_simplex(f_coords)
                
                if simplex != -1:                    
                    min_dist_s = f
              
                
    # ищем расстояние до каждого ребра через функцию distance_point_to_edge 
    for i in range(len(edge_central_coords)):
        digits = edge_central_coords[i]
        for var in itertools.combinations(digits, r=2):
            dist_point = distance_point_to_edge(s, var[0], var[1])

            # если полученное расстояние меньше минимального, то меняем значение минимального расстояния
            if dist_point < min_dist_s:
                min_dist_s = dist_point

                
    return sub_grid_0, ((min_dist_s * 2)/ max_len)

In [ ]:
# берем за основу матрицу подрешетки:
  #  [[ 0,  0,  0,  0],
   #  [ 0,  1, -2,  1],
    # [ 0, -1, -1,  2],
     #[ 0,  2,  3,  1]]
# и перебираем матрицы, ставя вместо нулей числа: [-3, -2, -1, 0, 1, 2, 3]

mat = np.array([[0, 0, 0, 0], [0, 1, -2, 1], [0, -1, -1, 2], [0, 2, 3, 1]])
list_mat = []
list_ch = []
list_dist = []
list_val = [-3, -2, -1, 0, 1, 2, 3]
for i1 in list_val:
    mat[3][0] = i1
    for i2 in list_val:
        mat[2][0] = i2
        for i3 in list_val:
            mat[1][0] = i3
            for i4 in list_val:
                mat[0][0] = i4
                for i5 in list_val:
                    mat[0][1] = i5
                    for i6 in list_val:
                        mat[0][2] = i6
                        for i7 in list_val:
                            mat[0][3] = i7
                            det_mat = abs(round(np.linalg.det(mat), 0))
                            
                            # если модуль определителя от 26 до 81, то считаем запрещенное расстояние
                            if det_mat >= 26 and det_mat < 81:
                                sub_grid_0 = np.array(mat)
                                sub_grid_good, forb_dist = ch_number_forb_segm(grid, mat)
                              
                                # если запрещенное расстояние больше 1, то записываем матрицу в список 
                                # подходящих матриц
                                if forb_dist > 1:
                                    list_mat.append(sub_grid_good)
                                    list_ch.append(det_mat)
                                    list_dist.append(forb_dist)
                                    print(sub_grid_good, det_mat, forb_dist, 'OK')
    print(i1)